In [ ]:
info_path = 'C:/Users/xyttttt/Desktop/info.csv'
out_path = 'C:/Users/xyttttt/Desktop/kill_sale.csv'

import numpy as np
import pandas as pd
from pandas import DataFrame,Series
import matplotlib.pyplot as plt #导入图像库
import matplotlib
import datetime
plt.rcParams['font.sans-serif'] = ['SimHei'] #用来正常显示中文标签，黑体
plt.rcParams['axes.unicode_minus'] = False #用来正常显示负号

allDataDate=[]
days = [31,28,31,30,31,30,31,31,30,31,30,31]
for y in [2017,2018]:
    for m in range(1,13):
        for d in range(1,days[m-1]+1):
            allDataDate.append(int(str(y)+('0'+str(m))[-2:]+('0'+str(d))[-2:]))
allDataDate = allDataDate[59:520]
allDate = {}
wd = []
weekday = 1
weeknum = 1
for date in allDataDate:
    if(weekday%7 == 0):
        weeknum+=1
    allDate[date] = weeknum
    wd.append(weekday%7+1)
    weekday+=1
allDate = pd.DataFrame({'data_date':list(allDate.keys()),'week_num':list(allDate.values()),'week_day_num':wd})
allDate['day_num'] = allDate.index+1

date1 = allDate.iloc[0:440-59,:].copy()

date2 = allDate.iloc[485-59:520-59,:].copy()

date3 = pd.concat([date1,date2])

date3 = date3.reset_index().drop(['index','week_day_num','day_num'],axis=1)

info_t = pd.read_csv(info_path)

info_t = info_t[['goods_id','goods_season']].copy()

def ff1(num):
    if num==0:
        return 15
    elif num==1:
        return 8
    elif num==2:
        return 4
    elif num==3:
        return 2
    elif num==4:
        return 1
    elif num==5:
        return 12
    elif num==6:
        return 10
    elif num==7:
        return 9
    elif num==8:
        return 6
    elif num==9:
        return 5
    else :
        return 3
    
info_t.loc[:,'season_list'] = info_t['goods_season'].agg(ff1)

dd = date3.copy()

dd.loc[:,'date_time'] = pd.to_datetime(dd.data_date.apply(str))

def mymonth(dt):
    return dt.month

dd.loc[:,'month'] = dd.date_time.agg(mymonth)

def ff2(a):
    if a>=3 and a<=5:
        return 8
    elif a>=6 and a<=8:
        return 4
    elif a>=9 and a<=11:
        return 2
    else:
        return 1
    
dd.loc[:,'monthlist'] = dd['month'].agg(ff2)

dd = dd[['week_num','monthlist']].copy()

info_t = info_t[['goods_id','season_list']].copy()

dd.loc[:,'tt'] = 1
info_t.loc[:,'tt'] = 1
ss = pd.merge(info_t,dd,on='tt').drop('tt',axis=1)

sss = ss[['goods_id','week_num','season_list','monthlist']].copy()
sss.loc[:,'ttt'] = 0

def fff(num):
    t = 0
    while(num!=0):
        num &= (num-1)
        t += 1
    return t

sss['ttt'] = sss['monthlist'] & sss['season_list']

sss['ttt'] = sss['ttt'].agg(fff)

out = sss.groupby(['goods_id','week_num'],as_index=False)['ttt'].mean()

out.to_csv(out_path,index=False)